In [ ]:
import kfp
from kfp import dsl
from kfp import kubernetes

@dsl.component(
    base_image='traininghost/pipelinegpuimage:latest',
)
def gpu_smoking_check():
    import subprocess
    result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    print(result.stdout)

@dsl.pipeline(
    name='GPU smoke check',
    description='Smoke check to see whether GPU environment is ready.'
)
def gpu_pipeline():
    gpu_check_task = gpu_smoking_check() \
        .set_gpu_limit(1)
    kubernetes.set_image_pull_policy(gpu_check_task, "IfNotPresent")

if __name__ == '__main__':
    file_name = "gpu_smoke_check"
    kfp.compiler.Compiler().compile(gpu_pipeline, f'{file_name}.yaml')

In [ ]:
import requests
pipeline_name = "gpu_smoke_check"
pipeline_file = file_name + '.yaml'
response = requests.post(
    "http://tm.traininghost:32002/pipelines/{}/upload".format(pipeline_name),
    files={'file': open(pipeline_file,'rb')}
)

if response.status_code == 200:
    print(f"Pipeline YAML file '{file_name}.yaml' has been generated and uploaded successfully.")
else:
    print(f"Failed to upload pipeline. Status code: {response.status_code}")
    print(f"Response content: {response.content}")